In [ ]:
import numpy as np
import ast
import math
import random
import logging

from Robot import Robot

# Policy
The implementation of a policy for the robot. It can be a simple path following policy for a robot, or it can encompass something complex, like an RL model.

In [ ]:
class Policy:
    def __init__(self, name, env, robot):
        self.name = name
        self.env = env
        self.robot = robot
        
    def act(self, delta_t):
        """Act over the time of delta T. When implemented, it should add an action to be executed to
        the robot's action queue - it is not supposed to execute it here"""
        pass

In [ ]:
class AbstractWaypointPolicy(Policy):
    """The ancestor of all the policies that are based on choosing waypoints"""
    
    ##def __init__(self, name, env, robot):
    ##    super().__init__("AbstractWaypointPolicy", env, robot)
    
    def move_towards_location(self, targetx, targety, vel, delta_t):
        """Schedule the actions to move the robot towards a target location. Returns true 
        if the robot will reach that location as the result of this step and false otherwise"""
        deltax = targetx - self.robot.x
        deltay = targety - self.robot.y
        veclength = np.linalg.norm([deltax, deltay])
        logging.debug(f"veclength {veclength}")
        if veclength == 0: # if we are there, stay there 
            self.robot.add_action("vel [0, 0]")
            return True
        if veclength <= self.vel * delta_t: # if we are closer than the velocity, then just go there
            logging.debug(f"We are there so go there [{targetx}, {targety}]")
            self.robot.add_action("vel [0, 0]")
            self.robot.add_action(f"loc [{targetx}, {targety}]")
            return True
        # move towards the location with the specified velocity
        #vel_x = min(vel * deltax / veclength, delta_t * deltax)
        #vel_y = min(vel * deltay / veclength, delta_t * deltay)
        vel_x = vel * deltax / veclength
        vel_y = vel * deltay / veclength
        action = f"vel [{vel_x}, {vel_y}]"
        logging.debug(f"move_towards_location setting action = {action}")
        self.robot.add_action(action)
        return False

In [ ]:
class GoToLocationPolicy(AbstractWaypointPolicy):
    """This is a simple policy that make the robot go to a certain location."""
    def __init__(self, env, robot, locx, locy, vel):
        super().__init__("GoToPolicy", env, robot)
        self.locx, self.locy, self.vel = locx, locy, vel
        
    def act(self, delta_t):
        """ Head towards locx, locy with a velocity vel"""
        ##logging.debug("HERE")
        self.move_towards_location(self.locx, self.locy, self.vel, delta_t)

In [ ]:
class FollowPathPolicy(AbstractWaypointPolicy):
    """A policy that makes a robot follow a certain path specified as a series of waypoints. 
    If repeat is true, it repeats the path indefinitely."""
    def __init__(self, env, robot, vel, waypoints, repeat = False):
        super().__init__("FollowPathPolicy", env, robot)
        self.waypoints = waypoints.copy()
        self.vel = vel
        self.currentwaypoint = 0
        self.repeat = repeat
        
    def act(self, delta_t):
        ## if we are done with the movement, don't do anything
        if self.currentwaypoint == -1:
            return
        ## move towards the current waypoint
        wp = self.waypoints[ self.currentwaypoint ]
        done = self.move_towards_location(wp[0], wp[1], self.vel, delta_t)
        if done:
            self.currentwaypoint = self.currentwaypoint + 1
            if self.currentwaypoint == len(self.waypoints):
                if self.repeat: 
                    self.currentwaypoint = 0
                else:
                    self.currentwaypoint = -1

In [ ]:
class RandomWaypointPolicy(AbstractWaypointPolicy):
    """A policy that makes the robot follow a random waypoint behavior within a specified region
    using a constant velocity.
    The region is specied with the low_point and high_point each of them having (x,y) formats """
    def __init__(self, env, robot, vel, low_point, high_point):
        super().__init__("RandomWaypointPolicy", env, robot)
        self.vel = vel
        self.low_point = low_point
        self.high_point = high_point
        self.nextwaypoint = None
                
    def act(self, delta_t):
        """Moves towards the chosen waypoint. If the waypoint is """
        if self.nextwaypoint == None:
            x = random.uniform(self.low_point[0], self.high_point[0])
            y = random.uniform(self.low_point[1], self.high_point[1])
            self.nextwaypoint = [x, y]
        done = self.move_towards_location(self.nextwaypoint[0], self.nextwaypoint[1], self.vel, delta_t)
        if done:
            self.nextwaypoint = None

## Experiments for trying out the different policies

In [ ]:
robot = Robot("Robi", 20, 30, 0)
robot.policy = GoToLocationPolicy(None, robot, 40, 40, 0.6)
#print(robot)
#for i in range(40):
#    robot.enact_policy()
#    robot.proceed()
#    print(robot)

print("-------------- Robur -------------------")
robot = Robot("Robur", 4, 8, 0)
robot.policy = RandomWaypointPolicy(None, robot, 1, [0,0], [10, 10])
print(robot)
for i in range(40):
    robot.enact_policy()
    robot.proceed()
    print(robot)


robot = Robot("R2D3", 5, 9, 0)
robot.policy = FollowPathPolicy(None, robot, 1, [[0,0], [5, 5], [10,0]], repeat = True)
    